In [13]:
import tensorflow as tf

PATCH_SIZE = 7
PATCH_PIXELS = 7*7

BATCH_SIZE = 100

NUM_THREADS = 1

def read(queue):
    reader = tf.TFRecordReader()
    _, example = reader.read(queue)
    
    features = tf.parse_single_example(example, features={
        'us_patch': tf.FixedLenFeature([], tf.string),
        'mr_patch': tf.FixedLenFeature([], tf.string),
    })
    
    image = tf.decode_raw(features['mr_patch'], tf.float64)
    label = tf.decode_raw(features['us_patch'], tf.float64)
    
    image.set_shape([PATCH_PIXELS])
    label.set_shape([PATCH_PIXELS])
    
    return image, label

def inputs(filenames):
    queue = tf.train.string_input_producer(filenames)
    
    return tf.train.shuffle_batch(read(queue),
        batch_size=BATCH_SIZE,
        num_threads=NUM_THREADS,
        capacity=1000 + 3*BATCH_SIZE,
        min_after_dequeue=1000)

In [17]:
# weights and biases
w = tf.Variable(tf.zeros([PATCH_PIXELS, PATCH_PIXELS], tf.float64))
b = tf.Variable(tf.zeros([PATCH_PIXELS], tf.float64))

# placeholders to be fed with images, labels batch
x = tf.placeholder(tf.float64, shape=[None, PATCH_PIXELS])
y = tf.placeholder(tf.float64, shape=[None, PATCH_PIXELS])

# prediction function
f = tf.matmul(x, w) + b

# cost function
cost = tf.reduce_mean(tf.square(f-y), 1)

# train step
train = tf.train.GradientDescentOptimizer(.5).minimize(cost)

In [20]:
pred = tf.equal(tf.argmax(y, 1), tf.argmax(f, 1))
loss = tf.reduce_mean(tf.cast(pred, tf.float32))

In [28]:
import time

test_files = ['th-30/{:02}.tfrecord'.format(i) for i in range(1, 8)]
train_files = ['th-30/{:02}.tfrecord'.format(i) for i in range(8, 15)]

images, labels = inputs(train_files)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    try:
        step = 0
        
        while not coord.should_stop():
            start_time = time.time()
        
            _, accuracy = sess.run([train, loss], feed_dict={
                x: images.eval(), y: labels.eval()
            })
            
            duration_time = start_time - time.time()
            
            if step % 100 == 0:        
                print('step {} with accuracy {}'.format(step, accuracy))
            
            step += 1
    except tf.errors.OutOfRangeError:
        print('done with training')
    finally:
        coord.request_stop()
        coord.join(threads)

step 0 with accuracy 0.07000000029802322
step 100 with accuracy 0.029999999329447746
step 200 with accuracy 0.07999999821186066
step 300 with accuracy 0.20000000298023224
step 400 with accuracy 0.12999999523162842
step 500 with accuracy 0.07000000029802322
step 600 with accuracy 0.05999999865889549
step 700 with accuracy 0.07999999821186066
step 800 with accuracy 0.05000000074505806
step 900 with accuracy 0.029999999329447746
step 1000 with accuracy 0.019999999552965164
step 1100 with accuracy 0.05999999865889549
step 1200 with accuracy 0.07000000029802322
step 1300 with accuracy 0.12999999523162842
step 1400 with accuracy 0.019999999552965164
step 1500 with accuracy 0.019999999552965164
step 1600 with accuracy 0.05999999865889549
step 1700 with accuracy 0.05999999865889549
step 1800 with accuracy 0.029999999329447746
step 1900 with accuracy 0.009999999776482582
step 2000 with accuracy 0.029999999329447746
step 2100 with accuracy 0.07999999821186066
step 2200 with accuracy 0.1099999994

KeyboardInterrupt: 